In [2]:
!pip uninstall spacy
!pip uninstall spacy-transformers
!pip install spacy-transformers
!python -m spacy download en_trf_bertbaseuncased_lg

Uninstalling spacy-2.3.2:
  Would remove:
    /usr/local/bin/spacy
    /usr/local/lib/python3.6/dist-packages/bin/*
    /usr/local/lib/python3.6/dist-packages/spacy-2.3.2.dist-info/*
    /usr/local/lib/python3.6/dist-packages/spacy/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.6/dist-packages/bin/theano_cache.py
    /usr/local/lib/python3.6/dist-packages/bin/theano_nose.py
Proceed (y/n)? y
  Successfully uninstalled spacy-2.3.2
Uninstalling spacy-transformers-0.6.2:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/spacy_transformers-0.6.2.dist-info/*
    /usr/local/lib/python3.6/dist-packages/spacy_transformers/*
    /usr/local/lib/python3.6/dist-packages/tests/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.6/dist-packages/tests/download_media.py
    /usr/local/lib/python3.6/dist-packages/tests/test_AudioClips.py
    /usr/local/lib/python3.6/dist-packages/tests/test_ImageSequenceClip.py
    /usr/local/lib/python3.6/dist

✔ Download and installation successful
You can now load the model via spacy.load('en_trf_bertbaseuncased_lg')


In [2]:
!mkdir drug_data

In [5]:
import xml.etree.ElementTree as ET

In [6]:
import re
def to_dict(sent, index, type_g):
  s = len(type_g)
  tupl = (sent,)
  result = {}
  result['entities'] = []

  for i in range(s):
    if ';' in index[i]:
      index[i]=index[i].split(';')[0]
    begin = index[i].split('-')[0]
    end = index[i].split('-')[1]
    label = '%s' %type_g[i].upper()
    result['entities'].append((int(begin), int(end)+1, label))
  tupl+=(result,)
  return tupl

In [7]:
def xml_to_json(fin, result = []):
  tree = ET.parse(fin)
  root = tree.getroot()
  for neighbor in root.iter('sentence'):
    #print(neighbor.attrib.get('text'))
    text = neighbor.attrib.get('text')
    index = []
    tags = []
    values = []
    for each in neighbor:
      if each.tag == 'entity':
        index.append(each.attrib.get('charOffset'))
        tags.append(each.attrib.get('type'))
        #print(each.attrib.get('charOffset'), each.attrib.get('type'))
    result.append(to_dict(text, index, tags))
  return result
    

In [8]:
import glob
for file in glob.glob('drug_data/*.xml'):
  d = xml_to_json(file)
print(len(d))

5675


In [127]:
d[:20]

[('Food: Isoniazid should not be administered with food.',
  {'entities': [(6, 15, 'DRUG')]}),
 ('Studies have shown that the bioavailability of isoniazid is reduced significantly when administered with food.',
  {'entities': [(47, 56, 'DRUG')]}),
 ('Acetaminophen: A report of severe acetaminophen toxicity was reported in a patient receiving Isoniazid.',
  {'entities': [(0, 13, 'DRUG'), (34, 47, 'DRUG'), (93, 102, 'DRUG')]}),
 ('It is believed that the toxicity may have resulted from a previously unrecognized interaction between isoniazid and acetaminophen and a molecular basis for this interaction has been proposed.',
  {'entities': [(102, 111, 'DRUG'), (116, 129, 'DRUG')]}),
 ('However, current evidence suggests that isoniazid does induce P-450IIE1, a mixed-function oxidase enzyme that appears to generate the toxic metabolites, in the liver.',
  {'entities': [(40, 49, 'DRUG')]}),
 ('Furthermore it has been proposed that isoniazid resulted In induction of P-450IIE1 in the patients liv

In [48]:
nlp.make_doc(list(TRAIN_DATA[0])[0])

Cholestyramine and colestipol resins: Absorption of hydrochlorothiazide is impaired in the presence of anionic exchange resins.

In [129]:
import spacy
from spacy.util import minibatch
import random
import torch
import en_trf_bertbaseuncased_lg
TRAIN_DATA = d

is_using_gpu = spacy.prefer_gpu()
if is_using_gpu:
    torch.set_default_tensor_type("torch.cuda.FloatTensor")

#nlp = spacy.load("en_trf_bertbaseuncased_lg")
nlp = en_trf_bertbaseuncased_lg.load()
print(nlp.pipe_names) # ["sentencizer", "trf_wordpiecer", "trf_tok2vec"]
ner = nlp.create_pipe('ner')
nlp.add_pipe(ner, last=True)

# add labels
for _, annotations in TRAIN_DATA:
      for ent in annotations.get('entities'):
        ner.add_label(ent[2])

optimizer = nlp.begin_training()
for i in range(10):
    random.shuffle(TRAIN_DATA)
    losses = {}
    for batch in minibatch(TRAIN_DATA, size=8):
        texts, lbls = zip(*batch)
        nlp.update(texts, lbls, sgd=optimizer, losses=losses)
    print(i, losses)
nlp.to_disk("bert-ner")
#then load this one, then add pipe of the en web lg etc

['sentencizer', 'trf_wordpiecer', 'trf_tok2vec']


/usr/local/lib/python3.6/dist-packages/spacy/language.py:639: UserWarning: [W033] Training a new parser or NER using a model with no lexeme normalization table. This may degrade the performance of the model to some degree. If this is intentional or the language you're using doesn't have a normalization table, please ignore this warning. If this is surprising, make sure you have the spacy-lookups-data package installed. The languages with lexeme normalization tables are currently: da, de, el, en, id, lb, pt, ru, sr, ta, th.
  **kwargs
/usr/local/lib/python3.6/dist-packages/spacy_transformers/language.py:91: UserWarning: [W030] Some entities could not be aligned in the text "Aspirin/Nonsteroidal Antiinflammatory Drugs (NSAID..." with entities "[(0, 7, 'BRAND'), (8, 42, 'GROUP'), (45, 51, 'GROU...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
/usr/l

0 {'ner': 11887.703118033038}
1 {'ner': 4212.905950548706}
2 {'ner': 3160.2575170605787}
3 {'ner': 2352.4480103388787}
4 {'ner': 2011.3655015382508}
5 {'ner': 1832.1552733600288}
6 {'ner': 1596.1284652344539}
7 {'ner': 1440.327099336226}
8 {'ner': 1410.2672530432392}
9 {'ner': 1095.4263953945233}


In [134]:
# if 'ner' not in nlp.pipe_names:
#     ner = nlp.create_pipe('ner')
#     nlp.add_pipe(ner, last=True)
       
# else:
#     ner = nlp.get_pipe('ner')
#     # add labels
# nlp2 = en_core_web_lg.load()

# nlp.add_pipe(nlp2.create_pipe('tagger'), last=True)
# nlp.add_pipe(nlp2.create_pipe('parser'), last=True)
# nlp.begin_training()


ValueError: ignored

In [66]:
#nlp.to_disk("bert-ner")

In [131]:
!tar -czvf bert-ner.tar.gz bert-ner

bert-ner/
bert-ner/sentencizer.json
bert-ner/meta.json
bert-ner/ner/
bert-ner/ner/moves
bert-ner/ner/model
bert-ner/ner/cfg
bert-ner/vocab/
bert-ner/vocab/lookups.bin
bert-ner/vocab/strings.json
bert-ner/vocab/vectors
bert-ner/vocab/key2row
bert-ner/trf_wordpiecer/
bert-ner/trf_wordpiecer/model
bert-ner/trf_wordpiecer/cfg
bert-ner/trf_tok2vec/
bert-ner/trf_tok2vec/model
bert-ner/trf_tok2vec/cfg
bert-ner/tokenizer


In [ ]:
# test_text = input("Enter your testing text: ")
# doc = nlp(test_text)
# for ent in doc.ents:
#     print(ent)
#     print(ent.text, ent.start_char, ent.end_char, ent.label_)

In [32]:
x = [('Germany is one of the best countries in Europe',
  {'entities': [(0, 7, 'GPE'), (40, 46, 'LOC')]}),
 ('Bayer is a pharmaceutical company',
  {'entities': [(0, 5, 'ORG')]}),
 ('Werner Baumann has been CEO of Bayer since 2016',
  {'entities': [(0, 14, 'PERSON'), (31, 36, 'ORG'), (43, 47, 'DATE')]}),
 ('Monsanto has been aquired by Bayer',
  {'entities': [(0, 8, 'ORG'), (29, 34, 'ORG')]})]
d[:10]+x

[('Drug/Laboratory Test Interaction Because Xigris may affect the APTT assay, Xigris present in plasma samples may interfere with one-stage coagulation assays based on the APTT (such as factor VIII, IX, and XI assays).',
  {'entities': [(41, 47, 'BRAND'), (75, 81, 'BRAND')]}),
 ('Few systemic data have been collected on the metabolism of WELLBUTRIN following concomitant administration with other drugs or, alternatively, the effect of concomitant administration of WELLBUTRIN on the metabolism of other drugs.',
  {'entities': [(59, 69, 'BRAND'), (187, 197, 'BRAND')]}),
 ('Respiratory depression, hypotension, and profound sedation or coma may occur.',
  {'entities': []}),
 ('During administration of multiple oral doses of TAMBOCOR to healthy subjects stabilized on a maintenance dose of digoxin, a 13%-19% increase in plasma digoxin levels occurred at six hours postdose.',
  {'entities': [(48, 56, 'BRAND'), (113, 120, 'DRUG'), (151, 158, 'DRUG')]}),
 ('This effect has been attributed to inh

In [3]:
!python -m spacy download en_core_web_lg

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [46]:
import spacy
import random
import torch
import en_trf_bertbaseuncased_lg
import en_core_web_lg
from spacy.util import minibatch

x = [('Germany is one of the best countries in Europe',
  {'entities': [(0, 7, 'GPE'), (40, 46, 'LOC')]}),
 ('Bayer is a pharmaceutical company',
  {'entities': [(0, 5, 'ORG')]}),
 ('Werner Baumann has been CEO of Bayer since 2016',
  {'entities': [(0, 14, 'PERSON'), (31, 36, 'ORG'), (43, 47, 'DATE')]}),
 ('Monsanto has been aquired by Bayer',
  {'entities': [(0, 8, 'ORG'), (29, 34, 'ORG')]}),
  ('It used Aspirin widely',
  {'entities': [(8, 15, 'BRAND')]})]

def train_spacy(data,iterations):
    TRAIN_DATA = d
    nlp = en_core_web_lg.load()  
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
       
    else:
        ner = nlp.get_pipe('ner')

    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for i in range(iterations):
          random.shuffle(TRAIN_DATA)
          TRAIN_DATA+=x 
          losses = {}
          for batch in minibatch(TRAIN_DATA[:50], size=8):
              texts, lbls = zip(*batch)
              nlp.update(texts, lbls, sgd=optimizer, losses=losses)
          print(i, losses)
    return nlp

prdnlp = train_spacy(d, 10)

# Save our trained Model
#modelfile = input("Enter your Model Name: ")
#prdnlp.to_disk(modelfile)


#Test your text
test_text = input("Enter your testing text: ")
doc = prdnlp(test_text)
for ent in doc.ents:
    print(ent.label_, ent.text)

0 {'ner': 264.28702589683235}
1 {'ner': 166.71085100015625}
2 {'ner': 205.49982899124734}


/usr/local/lib/python3.6/dist-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Corticosteroids and Corticotropin (ACTH): may pote..." with entities "[(0, 15, 'GROUP'), (20, 33, 'DRUG'), (35, 39, 'GRO...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)


3 {'ner': 152.1479502679431}


/usr/local/lib/python3.6/dist-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "The extent to which SSRI-TCAinteractions may pose ..." with entities "[(20, 24, 'GROUP'), (25, 28, 'GROUP'), (140, 144, ...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
/usr/local/lib/python3.6/dist-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Antihypertensives: Amiodarone should be used with ..." with entities "[(0, 17, 'DRUG'), (19, 29, 'DRUG'), (82, 90, 'DRUG...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)


4 {'ner': 94.73906521631329}


/usr/local/lib/python3.6/dist-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Lithium generally should not be given with diureti..." with entities "[(0, 7, 'DRUG'), (43, 52, 'GROUP'), (73, 80, 'DRUG...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)


5 {'ner': 114.3987549964495}
6 {'ner': 115.5949237514925}


/usr/local/lib/python3.6/dist-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Ketoconazole: Ketoconazole may inhibit both synthe..." with entities "[(0, 12, 'DRUG'), (14, 26, 'DRUG'), (79, 88, 'GROU...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
/usr/local/lib/python3.6/dist-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "The concurrent use of two or more drugs with antic..." with entities "[(82, 100, 'GROUP'), (106, 120, 'DRUG'), (126, 147...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)


7 {'ner': 102.97414008354588}
8 {'ner': 121.6601889677986}
9 {'ner': 88.49441014978561}
('BRAND', 'CARDINAL', 'DATE', 'DRUG', 'DRUG_N', 'EVENT', 'FAC', 'GPE', 'GROUP', 'LANGUAGE', 'LAW', 'LOC', 'MONEY', 'NORP', 'ORDINAL', 'ORG', 'PERCENT', 'PERSON', 'PRODUCT', 'QUANTITY', 'TIME', 'WORK_OF_ART')
Enter your testing text: Aspirin is a brand
DRUG Aspirin


In [21]:
ner.labels

NameError: ignored

In [45]:
test_text = input("Enter your testing text: ")
doc = prdnlp(test_text)
for ent in doc.ents:
    print(ent.label_, ent.text)

Enter your testing text: Vietnam is in Asia
GPE Vietnam
LOC Asia


In [15]:
print(prdnlp.pipe_names)

['tagger', 'parser', 'ner']


In [139]:
!rm -r 'xxx'